In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


Import the necessary packages

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Reading the train dataset, test dataset and the target for the train dataset from the train_labels file

In [5]:
import os
os.chdir("/content/drive/My Drive/Project/DengAI")
os.listdir()

['dengue_labels_train.csv',
 'dengue_features_train.csv',
 'dengue_features_test.csv',
 'submission_format.csv',
 'DengAI -1.ipynb',
 'dengai-2.ipynb',
 'firstsubmission.csv',
 'DengAI_Sarimax_final.ipynb',
 'DengAI auto_arima.ipynb',
 '.ipynb_checkpoints',
 'secondsubmission.csv',
 'DengAI_time_series_Finalmentor.ipynb',
 'sarimaxsubmission.csv',
 'arimaxsubmission.csv']

In [0]:
train = pd.read_csv("dengue_features_train.csv")
train_labels = pd.read_csv("dengue_labels_train.csv")
test = pd.read_csv("dengue_features_test.csv")

In [7]:
print('Train dataset',train.shape)
print('Test dataset',test.shape)
print('Train_label ',train_labels.shape)

Train dataset (1456, 24)
Test dataset (416, 24)
Train_label  (1456, 4)


In [8]:
print('TRAIN HEAD')
train.head(5)

TRAIN HEAD


,city,year,weekofyear,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,297.572857,297.742857,292.414286,299.8,295.9,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,298.211429,298.442857,293.951429,300.9,296.4,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,298.781429,298.878571,295.434286,300.5,297.3,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,298.987143,299.228571,295.310000,301.4,297.0,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,299.518571,299.664286,295.821429,301.9,297.5,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8


In [0]:
print('TRAIN LABELS HEAD')
train_labels.head(5)

TRAIN LABELS HEAD


,city,year,weekofyear,total_cases
0,sj,1990,18,4
1,sj,1990,19,5
2,sj,1990,20,4
3,sj,1990,21,3
4,sj,1990,22,6


The submission format demands that there is city, year, week of year in the submission file. That is why we are storing them for future submission

In [0]:
test_city=test['city']
test_year=['year']
test_weekofyear=test['weekofyear']

In [10]:
train.describe()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
count,1456.000000,1456.000000,1262.000000,1404.000000,1434.000000,1434.000000,1443.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,1446.000000,1443.000000,1446.000000,1446.000000,1413.000000,1413.000000,1436.000000,1442.000000,1434.000000
mean,2001.031593,26.503434,0.142294,0.130553,0.203783,0.202305,45.760388,298.701852,299.225578,295.246356,303.427109,295.719156,40.151819,82.161959,45.760388,16.746427,4.903754,27.185783,8.059328,32.452437,22.102150,39.326360
std,5.408314,15.019437,0.140531,0.119999,0.073860,0.083903,43.715537,1.362420,1.261715,1.527810,3.234601,2.565364,43.434399,7.153897,43.715537,1.542494,3.546445,1.292347,2.128568,1.959318,1.574066,47.455314
min,1990.000000,1.000000,-0.406250,-0.456100,-0.015533,-0.063457,0.000000,294.635714,294.892857,289.642857,297.800000,286.900000,0.000000,57.787143,0.000000,11.715714,1.357143,21.400000,4.528571,26.700000,14.700000,0.000000
25%,1997.000000,13.750000,0.044950,0.049217,0.155087,0.144209,9.800000,297.658929,298.257143,294.118929,301.000000,293.900000,13.055000,77.177143,9.800000,15.557143,2.328571,26.300000,6.514286,31.100000,21.100000,8.700000
50%,2002.000000,26.500000,0.128817,0.121429,0.196050,0.189450,38.340000,298.646429,299.289286,295.640714,302.400000,296.200000,27.245000,80.301429,38.340000,17.087143,2.857143,27.414286,7.300000,32.800000,22.200000,23.850000
75%,2005.000000,39.250000,0.248483,0.216600,0.248846,0.246982,70.235000,299.833571,300.207143,296.460000,305.500000,297.900000,52.200000,86.357857,70.235000,17.978214,7.625000,28.157143,9.566667,33.900000,23.300000,53.900000
max,2010.000000,53.000000,0.508357,0.454429,0.538314,0.546017,390.600000,302.200000,302.928571,298.450000,314.000000,299.900000,570.500000,98.610000,390.600000,20.461429,16.028571,30.800000,15.800000,42.200000,25.600000,543.300000


In [11]:
test.describe()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,reanalysis_max_air_temp_k,reanalysis_min_air_temp_k,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
count,416.000000,416.000000,373.000000,405.000000,415.000000,415.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,414.000000,404.000000,404.000000,413.000000,407.000000,411.000000
mean,2010.766827,26.439904,0.126050,0.126803,0.207702,0.201721,38.354324,298.818295,299.353071,295.419179,303.623430,295.743478,42.171135,82.499810,38.354324,16.927088,5.124569,27.369587,7.810991,32.534625,22.368550,34.278589
std,1.434835,14.978257,0.164353,0.141420,0.079102,0.092028,35.171126,1.469501,1.306233,1.523099,3.101817,2.761109,48.909514,7.378243,35.171126,1.557868,3.542870,1.232608,2.449718,1.920429,1.731437,34.655966
min,2008.000000,1.000000,-0.463400,-0.211800,0.006200,-0.014671,0.000000,294.554286,295.235714,290.818571,298.200000,286.200000,0.000000,64.920000,0.000000,12.537143,1.485714,24.157143,4.042857,27.200000,14.200000,0.000000
25%,2010.000000,13.750000,-0.001500,0.015975,0.148670,0.134079,8.175000,297.751429,298.323214,294.335714,301.425000,293.500000,9.430000,77.397143,8.175000,15.792857,2.446429,26.514286,5.928571,31.100000,21.200000,9.100000
50%,2011.000000,26.000000,0.110100,0.088700,0.204171,0.186471,31.455000,298.547143,299.328571,295.825000,302.750000,296.300000,25.850000,80.330000,31.455000,17.337143,2.914286,27.483333,6.642857,32.800000,22.200000,23.600000
75%,2012.000000,39.000000,0.263329,0.242400,0.254871,0.253243,57.772500,300.240357,300.521429,296.643571,305.800000,298.275000,56.475000,88.328929,57.772500,18.174643,8.171429,28.319048,9.812500,33.900000,23.300000,47.750000
max,2013.000000,53.000000,0.500400,0.649000,0.453043,0.529043,169.340000,301.935714,303.328571,297.794286,314.100000,299.700000,301.400000,97.982857,169.340000,19.598571,14.485714,30.271429,14.725000,38.400000,26.700000,212.000000


In [0]:
sj_train_features=train.loc[train['city']=='sj']
sj_train_labels=train_labels.loc[train_labels['city']=='sj']
sj_test=test.loc[test['city']=='sj']

In [0]:
iq_train_features=train.loc[train['city']=='iq']
iq_train_labels=train_labels.loc[train_labels['city']=='iq']
iq_test=test.loc[test['city']=='iq']

Filling the missing values of continuous variable with median and categorical variable with most frequent value.

In [0]:
sj_train_features.fillna(method='ffill',inplace=True)
sj_test.fillna(method='ffill',inplace=True)
iq_train_features.fillna(method='ffill',inplace=True)
iq_test.fillna(method='ffill',inplace=True)


total_cases is our target variable and it is present in train_labels data frame

In [0]:
sj_target_variable=sj_train_labels.total_cases
iq_target_variable=iq_train_labels.total_cases

In [27]:
sj_target_variable.head()

0    4
1    5
2    4
3    3
4    6
Name: total_cases, dtype: int64

In [21]:
pd.isnull(sj_test).any()

city                                     False
year                                     False
weekofyear                               False
week_start_date                          False
ndvi_ne                                  False
ndvi_nw                                  False
ndvi_se                                  False
ndvi_sw                                  False
precipitation_amt_mm                     False
reanalysis_air_temp_k                    False
reanalysis_avg_temp_k                    False
reanalysis_dew_point_temp_k              False
reanalysis_max_air_temp_k                False
reanalysis_min_air_temp_k                False
reanalysis_precip_amt_kg_per_m2          False
reanalysis_relative_humidity_percent     False
reanalysis_sat_precip_amt_mm             False
reanalysis_specific_humidity_g_per_kg    False
reanalysis_tdtr_k                        False
station_avg_temp_c                       False
station_diur_temp_rng_c                  False
station_max_t

In [33]:
sj_train_features=sj_train_features.drop('city',axis=1)


KeyError: ignored

In [0]:
sj_train_features=sj_train_features.drop('week_start_date',axis=1)

In [0]:
sj_test=sj_test.drop('city',axis=1)
sj_test=sj_test.drop('week_start_date',axis=1)

In [0]:
iq_train_features=iq_train_features.drop('week_start_date',axis=1)
iq_train_features=iq_train_features.drop('city',axis=1)
iq_test=iq_test.drop('week_start_date',axis=1)
iq_test=iq_test.drop('city',axis=1)

Random Forest Model - sj

In [0]:
randForestModel = RandomForestRegressor(n_estimators=100, random_state=8)

In [35]:
randForestModel.fit(sj_train_features,np.ravel(sj_target_variable))

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=8, verbose=0, warm_start=False)

In [0]:
predictions = randForestModel.predict(sj_test)


In [0]:
predictions1=predictions.astype(int)

In [42]:
predictions1

array([  4,   5,   6,   7,   8,   8,   9,  12,  14,  14,  13,  16,  25,
        38,  55,  53,  63,  64, 122,  92,  79,  73,  77,  65,  74,  44,
        33,  50,  47,  33,  29,  27,  22,  21,  20,  13,  13,  13,  12,
        13,  10,  10,   8,   9,   4,   6,   6,   4,   5,   3,   4,   7,
         6,   6,   5,   5,   9,   7,   7,  11,   9,  15,  28,  23,  33,
        51,  55,  58,  66,  75, 121,  61,  61,  67,  86,  82,  80,  68,
        53,  41,  36,  69,  53,  34,  29,  30,  26,  26,  15,  13,  15,
        14,  11,  13,  12,  11,  11,   8,   6,  10,   7,   7,   4,   5,
        12,  12,   6,  12,  11,  28,  30,  34,  26,  21,  42,  42,  33,
        41,  53,  69,  77,  96, 113, 130, 106,  88,  77,  79, 104,  74,
        64,  47,  58,  26,  24,  20,  28,  21,  21,  20,  14,  13,  13,
        12,  12,  11,  11,   9,   8,   5,   6,   5,   5,   6,   5,   4,
         3,   6,   5,   5,  10,  12,  13,  28,  30,  27,  28,  27,  14,
        48,  42, 115,  73, 144,  78,  70,  72,  63,  79,  82,  7

Random Forest - IQ

In [0]:
randForestModel = RandomForestRegressor(n_estimators=100, random_state=8)

In [45]:
randForestModel.fit(iq_train_features,np.ravel(iq_target_variable))

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=8, verbose=0, warm_start=False)

In [0]:
predictions=randForestModel.predict(iq_test)

In [0]:
predictions2=predictions.astype(int)

In [48]:
predictions2

array([ 4,  3,  5,  3,  4,  4,  4,  4,  5,  8,  4,  4, 11,  7, 18, 11, 19,
        9, 10, 20, 16, 11, 12,  6, 16, 11, 12, 18, 14, 20, 18, 18, 18, 16,
       17, 15, 12, 11,  8,  7,  8,  7,  5,  5,  6,  2,  3,  6,  3,  3,  5,
        3,  2,  4,  2,  4,  2,  4,  4,  3,  3,  3,  3,  5,  7,  7,  7,  6,
       12, 23, 11, 13, 12, 11, 12, 12, 23, 23,  9, 17, 15, 18, 19, 16, 16,
       21, 16, 17, 10,  5,  6,  7,  5,  7,  7,  7,  4,  2,  8,  2,  3,  3,
        3,  4,  5,  3,  3,  6,  4,  2,  4,  4,  6,  3,  3,  5,  6, 11, 11,
        7, 10, 11, 11, 13, 18, 16, 12, 13, 10,  7, 14, 24, 21, 16, 21, 14,
       15, 15, 17, 11, 10, 12,  8,  8,  5,  8,  6,  5,  7,  4,  3,  5,  3,
        2,  3,  3])

In [0]:
submission = pd.read_csv('submission_format.csv',index_col=[0,1,2])

In [0]:
submission.total_cases=np.concatenate([predictions1,predictions2])

In [0]:
submission.to_csv('randforest1.csv')